# Notebook: Testing Our data_infra Package

## data_infra Package: PostgreSQL DB; AWS SQS, S3, etc.

In [21]:
# NOTE: First make sure you are in the TeamReel DS parent directory 
# (not the 'notebooks_modeling' directory):

In [22]:
ls

LICENSE                              dummy-test-api/
README.md                            notebook_data_infra_functions.ipynb
data_infra/                          notebooks_modeling/


------------------------

### get_next_video(): Database Info & Raw Video File

Gets the next newly uploaded video from our queue, downloads it to the project folder, queries our database for info about that video, prompt and user, and returns that info in a Python dictionary.

In [23]:
# Import:
# (Note: Make sure you are in the TeamReel DS parent directory, not 'notebooks_modeling'.)
from data_infra import get_next_video

In [24]:
help(get_next_video)

Help on function get_next_video in module data_infra.data_pipelines:

get_next_video()
    Gets the next newly uploaded video from our queue, downloads it to the
    project folder, queries our database for information about that video,
    prompt and user, and returns that info in a Python dictionary.
    
    (1) Checks our TeamReel AWS SQS queue for new messages (our S3 bucket
    sends a new message to SQS everytime any user uploads a new video);
    (2) Downloads the raw video file from S3 to your project directory; and
    (3) Gets relevant info about that video, prompt and user from our PostgreSQL DB.
    
    Returns
    -------
    - If queue has >=1 messages in it: A dictionary with all relevant info
    about that video, prompt, and user from our DB.
    - If queue is empty: String: "No messages in queue."



In [25]:
video_info = get_next_video()
video_info

{'video': {'video_id': 115,
  'title': 'HELLO WORLD',
  's3_key': 'videos/ALPACAVID-wHgVXLxaK.webm',
  's3_filename': 'ALPACAVID-wHgVXLxaK.webm',
  'created_at': datetime.datetime(2020, 3, 19, 20, 24, 24, 129000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
  'updated_at': datetime.datetime(2020, 3, 19, 20, 24, 24, 129000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))},
 'prompt': {'prompt_id': 45,
  'question': 'What is you cant think of a question to ask? is there somewhere where you can go to for examples?'},
 'user': {'user_id': 143,
  'first_name': 'Carlos',
  'last_name': 'Hernandez',
  'name': 'Carlos Hernandez',
  'username': 'carloshernandez'}}

In [35]:
# And now the raw video file is in our project folder too:

In [26]:
ls

ALPACAVID-wHgVXLxaK.webm             dummy-test-api/
LICENSE                              notebook_data_infra_functions.ipynb
README.md                            notebooks_modeling/
data_infra/


------------------

### get_video_info(video_s3_key=______): Database Info Only

Gets the video, prompt and user info for the specified video (using the input video_s3_key string) from our DB, and returns it in a Python dictionary. Does NOT download the video file.

In [29]:
# Import:
# (Note: Make sure you are in the TeamReel DS parent directory, not 'notebooks_modeling'.)
from data_infra import get_video_info

In [34]:
help(get_video_info)

Help on function get_video_info in module data_infra.postgresql_db_functions:

get_video_info(video_s3_key: str)
    Using the video's S3 key, looks up info for that video, user and prompt
    in our PostgreSQL database, and returns all info in a dictionary with
    the following top-level keys:
    {
        'video': {
            info for the video itself
        },
        'prompt': {
            info for the prompt the video was responding to
        },
        'user': {
            includes info for the user who posted the video
        }
    }



In [37]:
video_info = get_video_info(video_s3_key='videos/ALPACAVID-i7swK-Wzc.webm')
video_info

{'video': {'video_id': 134,
  'title': 'Kyla Oyamot',
  's3_key': 'videos/ALPACAVID-i7swK-Wzc.webm',
  's3_filename': 'ALPACAVID-i7swK-Wzc.webm',
  'created_at': datetime.datetime(2020, 5, 5, 16, 22, 56, 852000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
  'updated_at': datetime.datetime(2020, 5, 5, 16, 22, 56, 852000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))},
 'prompt': {'prompt_id': 65,
  'question': 'What is your first impression of Labs'},
 'user': {'user_id': 185,
  'first_name': 'Kyla',
  'last_name': 'O',
  'name': 'Kyla O',
  'username': 'kylao'}}

---------------------------

### get_feedback_for_video(video_id=_____): All feedback on that video

In [31]:
# Import:
# (Note: Make sure you are in the TeamReel DS parent directory, not 'notebooks_modeling'.)
from data_infra import get_feedback_for_video

In [32]:
help(get_feedback_for_video)

Help on function get_feedback_for_video in module data_infra.postgresql_db_functions:

get_feedback_for_video(video_id: int)
    Using the video's video_id in our database, looks up and
    returns all feedback on that video.



In [38]:
get_feedback_for_video(video_id=134)

,id,post,video_id,owner_id,created_at,updated_at,viewed
0,135,this is cool!,134,191,2020-05-05 16:23:30.411000+00:00,2020-05-05 16:23:30.411000+00:00,True
1,136,Great eye contact!,134,199,2020-05-05 16:23:37.899000+00:00,2020-05-05 16:23:37.899000+00:00,True
2,137,Feedback,134,192,2020-05-05 16:23:52.631000+00:00,2020-05-05 16:23:52.631000+00:00,True
3,138,feedback,134,200,2020-05-05 16:24:16.773000+00:00,2020-05-05 16:24:16.773000+00:00,True
4,139,feedback\n,134,193,2020-05-05 16:24:44.452000+00:00,2020-05-05 16:24:44.452000+00:00,True


----------------------------

### get_feedback_for_user(user_id=int): All feedback on all of that user's videos

In [39]:
# Import:
# (Note: Make sure you are in the TeamReel DS parent directory, not 'notebooks_modeling'.)
from data_infra import get_feedback_for_user

In [40]:
help(get_feedback_for_user)

Help on function get_feedback_for_user in module data_infra.postgresql_db_functions:

get_feedback_for_user(user_id: int)
    Using the user's user_id in our database (= 'owner_id' in the 'feedback' table),
    looks up and returns all feedback on all of that user's videos.



In [41]:
get_feedback_for_user(user_id=185)

,id,post,video_id,owner_id,created_at,updated_at,viewed
0,134,"If you did nothing, nothing would have been done.",130,185,2020-04-29 19:51:30.622000+00:00,2020-04-29 19:51:30.622000+00:00,True
1,140,Great eye contact!,135,185,2020-05-05 21:35:28.962000+00:00,2020-05-05 21:35:28.962000+00:00,True
